In [1]:
import time
import random
import threading
from threading import Thread

The most important class in the threading module is: Thread (doh!).

Very simplified, this is how a thread is instantiated:



In [ ]:
class Thread:
    def __init__(self, target, name=None, args=(), kwargs={}):
        pass

(there's a group argument which should be always None, as it's reserved for future use)

In this case, target is the function that will be executed in that particular thread.

Once a thread has been created (instantiated), we'll need to start() it in order for it to begin to process.

## Basic example of a thread

In [5]:
def simple_worker():
    print('hello', flush=True)
    time.sleep(5)
    print('world', flush=True)

In [6]:
t1 = Thread(target=simple_worker)

In [7]:
t1.start()

hello


In [8]:
2+4

6

world


In [9]:
t1.is_alive()

False

## Running multiple threads in parallel¶


In [11]:
def simple_worker():
    time.sleep(random.random() * 5)
    value = random.randint(0, 99)
    print(f'My value: {value}')

In [12]:
threads = [Thread(target=simple_worker) for _ in range(5)]

In [14]:

[t.start() for t in threads]

[None, None, None, None, None]

My value: 13
My value: 55
My value: 69
My value: 31
My value: 44


In [15]:
[t.join() for t in threads]

[None, None, None, None, None]

## Thread States¶


In [16]:
def simple_worker():

    print('Thread running...')
    time.sleep(5)
    print('Thread finished...')

In [17]:
t = Thread(target=simple_worker)

In [18]:
t.is_alive()

False

In [19]:

t.start()

Thread running...
Thread finished...


In [20]:

t.is_alive()

False

In [21]:

t.start()

RuntimeError: threads can only be started once

Important: It's not possible(*) to manage thread states manually, for example, stopping a thread. A thread always has to run its natural cycle.

(*) You might find hacks in the internet on how to stop threads, but it's a bad practice. We'll discuss more later.

## Thread Identity¶


In [22]:
def simple_worker():
    print('Thread running...')
    time.sleep(5)
    print('Thread exiting...')

In [23]:
t = Thread(target=simple_worker)

In [24]:
t.name

'Thread-14'

In [25]:
t.ident is None

True

In [26]:
t.start()

Thread running...


In [27]:
t.name

'Thread-14'

In [28]:
t.ident

16756

Thread exiting...


We can create a thread and assign a custom name to it:

In [29]:
t = Thread(target=simple_worker, name='PyCon 2020 Tutorial!')

In [30]:
t.start()

Thread running...


In [31]:
t.name

'PyCon 2020 Tutorial!'

Thread exiting...


In [32]:
t.ident

27096

## A thread knows itself

It's also possible to know the identity of the thread from within the thread itself. It might be counter intuitive as we don't have the reference to the created object, but the module function threading.currentThread() will provide access to it.

In [33]:
def simple_worker():
    sleep_secs = random.randint(1, 5)
    myself = threading.current_thread()
    ident = threading.get_ident()
    print(f"I am thread {myself.name} (ID {ident}), and I'm sleeping for {sleep_secs}.")
    time.sleep(sleep_secs)
    print(f'Thread {myself.name} exiting...')

In [36]:

t1 = Thread(target=simple_worker, name='Bubbles')
t2 = Thread(target=simple_worker, name='Blossom')
t3 = Thread(target=simple_worker, name='Buttercup')

In [37]:
t1.start()
t2.start()
t3.start()

I am thread Bubbles (ID 13616), and I'm sleeping for 3.
I am thread Blossom (ID 27236), and I'm sleeping for 2.
I am thread Buttercup (ID 25272), and I'm sleeping for 3.
Thread Blossom exiting...
Thread Bubbles exiting...
Thread Buttercup exiting...


## Passing parameters to threads

Passing parameters is simple with the thread constructor, just use the args argument:

In [38]:
def simple_worker(time_to_sleep):
    myself = threading.current_thread()
    ident = threading.get_ident()
    print(f"I am thread {myself.name} (ID {ident}), and I'm sleeping for {time_to_sleep}.")
    time.sleep(time_to_sleep)
    print(f'Thread {myself.name} exiting...')

In [39]:
t1 = Thread(target=simple_worker, name='Bubbles', args=(3, ))
t2 = Thread(target=simple_worker, name='Blossom', args=(1.5, ))
t3 = Thread(target=simple_worker, name='Buttercup', args=(2, ))

In [40]:
t1.start()
t2.start()
t3.start()

I am thread Bubbles (ID 1472), and I'm sleeping for 3.
I am thread Blossom (ID 13932), and I'm sleeping for 1.5.
I am thread Buttercup (ID 9512), and I'm sleeping for 2.
Thread Blossom exiting...
Thread Buttercup exiting...
Thread Bubbles exiting...


## Subclassing Thread


So far, the way we've created threads is by passing a target function to be executed. There's an alternative, more OOP-way to do it, which is extending the Thread class:



In [41]:
class MyThread(Thread):
    def __init__(self, time_to_sleep, name=None):
        super().__init__(name=name)
        self.time_to_sleep = time_to_sleep
        
    def run(self):
        ident = threading.get_ident()
        print(f"I am thread {self.name} (ID {ident}), and I'm sleeping for {self.time_to_sleep} secs.")
        time.sleep(self.time_to_sleep)
        print(f'Thread {self.name} exiting...')

In [42]:
t = MyThread(2)

In [43]:
t.run()

I am thread Thread-15 (ID 4420), and I'm sleeping for 2 secs.
Thread Thread-15 exiting...



## Shared Data

As we'll see, Threads can access shared data within the process they live in. Example:

In [44]:
TIME_TO_SLEEP = 2

In [45]:
def simple_worker():
    myself = threading.current_thread()
    print(f"I am thread {myself.name}, and I'm sleeping for {TIME_TO_SLEEP}.")
    time.sleep(TIME_TO_SLEEP)
    print(f'Thread {myself.name} exiting...')

In [46]:
t1 = Thread(target=simple_worker, name='Bubbles')
t2 = Thread(target=simple_worker, name='Blossom')
t3 = Thread(target=simple_worker, name='Buttercup')

In [47]:
t1.start()
t2.start()
t3.start()

I am thread Bubbles, and I'm sleeping for 2.
I am thread Blossom, and I'm sleeping for 2.
I am thread Buttercup, and I'm sleeping for 2.
Thread Bubbles exiting...Thread Blossom exiting...

Thread Buttercup exiting...


How is this possible?

Remember, all threads live within the same process, and the variable TIME_TO_SLEEP is stored in the process. So all the threads created can access that variable.

![title](thread_shared_data.png)